In [34]:
# Read Data
def read_data(filename):
	filename = "data/" + filename
	print("####################### STARCUBING FOR -> %s #######################" %filename) 
	f = open(filename, "r")
	fData = f.readlines()
	lData = []

	for sIter in fData:
		if (sIter[-1] == "\n"): # remove \n
			sIter = sIter[:-1]
		if (sIter == ""):
			continue
		lData.append(sIter.split(","))
	print("No of Features : %s" %(len(lData[0])))
	print("No of Samples: %s" %(len(lData)))
	return lData

# Create HashMaps
def create_dictionary(lData):
	lDicts = [dict() for x in range(len(lData[0]))] # list of dictionaries (Hashmaps)
	for i in range(0, len(lData[0])):
		dFeatures = {} # Empty hashmaps
		for j in range(0, len(lData)):
			if lData[j][i] in dFeatures.keys():
				dFeatures[lData[j][i]] = dFeatures[lData[j][i]] + 1
			else:
				dFeatures[lData[j][i]] = 1
		lDicts[i] = dFeatures
	return lDicts

def star_reduction(lData, lDicts, nIcebergCondn):
	lData1 = deepcopy(lData)
	dStar_Table = {}
	for i in range(0, len(lData[0])):
		dInterest = lDicts[i]
		for j in range(0, len(lData)):
			if (dInterest[lData[j][i]] < nIcebergCondn):
				dStar_Table[lData1[j][i]] = "*"
				lData1[j][i] = '*';
	return lData1

def tuple_convert(lData2):
	for i in range(0, len(lData2)):
		lData2[i] = tuple(lData2[i])
	return lData2

def generate_compressed_table(lData2):
	dFeatures = {}
	for i in range(0, len(lData2)):
		if lData2[i] in dFeatures.keys():
			dFeatures[lData2[i]] = dFeatures[lData2[i]] + 1
		else:
			dFeatures[lData2[i]] = 1
	return dFeatures

def lexographical_Sort_keys(dFeatures):
	lKeys = []
	lKeys = sorted(dFeatures.keys())
	return lKeys

#Tree class
class Tree(object):
	def __init__(node, label = "root", children = None, count = 0):
		node.name = label
		node.children = []
		node.count = count
		node.sibling = []
		if (children != None):
			for i in (0, len(children)):
				node.add_node(children[i])
	
	def add_node(node, child):
		node.children.append(child)
	
	def find(node, label = str("")):
		for i in range(0, len(node.children)):
			if (node.children[i].name == label):
				return node.children[i]
		return None
	
	def update_count(node, count = 0):
		node.count += count
	
	def add_sibling(node, sibling):
		node.sibling.append(sibling)

def create_star_tree(lKeys, dFeatures):
	star_tree = Tree()
	for i in range (0, len(lKeys)):
		head = star_tree
		add_children_to_tree(head, lKeys[i], 0, dFeatures)
	for i in range(0, len(star_tree.children)):
		star_tree.count += star_tree.children[i].count
	return star_tree

def add_children_to_tree(head, tChars, nIndex, dFeatures):
	tNew = head.find(tChars[nIndex])
	if (tNew != None):
		tNew.update_count(count = dFeatures[tChars])
	else:
		tNew = Tree(label = tChars[nIndex], count = dFeatures[tChars])
		head.add_node(tNew)
		if (len(head.children) > 1):
			head.children[len(head.children) - 2].add_sibling(tNew)
	nIndex += 1
	if (nIndex < len(tChars)):
		add_children_to_tree(tNew, tChars, nIndex, dFeatures)

def fPrintTree(tree):
	nIndex = 0
	if (len(tree.children) != 0):
		for child in tree.children:
			fPrintTree(tree.children[nIndex])
			nIndex+=1
	print(tree.name, tree.count)

def starcubing(star_tree, cNode, nLevel, lCuboidValList, dFeatures, lSubTrees):
	Cc = None
	lSubTrees[nLevel] = cNode
	if (cNode.count >= nIcebergCondn):
		if ((cNode.name != "root") or (len(cNode.children) == 0)):
			lCuboidValList[nLevel] = cNode.count
			print(lCuboidValList) # <-- UnComment this line to view the cuboid values
		else:
			Cc = Tree(label = "root", count = cNode.count)
	
	if (len(cNode.children) != 0):
			starcubing(star_tree, cNode.children[0], nLevel + 1, lCuboidValList, dFeatures, lSubTrees)
	if (Cc != None):
			starcubing(Cc, Cc, nLevel, lCuboidValList, dFeatures, lSubTrees)
	if (len(cNode.sibling) > 0):
			cNode.sibling[0].count += cNode.count
			starcubing(star_tree, cNode.sibling[0], nLevel,  lCuboidValList, dFeatures, lSubTrees)
	lCuboidValList[nLevel] = "*"


def write_results_to_csv(lDataCSV):
	sOutput = ""
	file = open('output1.csv','w') 
	for i in range(len(lDataCSV)):
		for j in range(len(lDataCSV[0])):
			sOutput += str(lDataCSV[i][j])
			sOutput += ","
		sOutput += '\n'
	file.write(sOutput)
    
nIcebergCondn = 2
from copy import deepcopy
import time
import psutil
import os

In [35]:
# Reading the file and creating list of lists
file = "validationDataset.txt"
lData = read_data(file)
lData

####################### STARCUBING FOR -> data/validationDataset.txt #######################
No of Features : 4
No of Samples: 5


[['a1', 'b1', 'c1', 'd1'],
 ['a1', 'b1', 'c4', 'd3'],
 ['a1', 'b2', 'c2', 'd2'],
 ['a2', 'b3', 'c3', 'd4'],
 ['a2', 'b4', 'c3', 'd4']]

In [36]:
# Generation of HashMaps and star reduction performed
lDicts = create_dictionary(lData)
lData1 = star_reduction(lData, lDicts, nIcebergCondn)
print("HashMaps:-")
print(lDicts)
print("Database replaced with star:-")
print(lData1)

HashMaps:-
[{'a1': 3, 'a2': 2}, {'b1': 2, 'b2': 1, 'b3': 1, 'b4': 1}, {'c1': 1, 'c4': 1, 'c2': 1, 'c3': 2}, {'d1': 1, 'd3': 1, 'd2': 1, 'd4': 2}]
Database replaced with star:-
[['a1', 'b1', '*', '*'], ['a1', 'b1', '*', '*'], ['a1', '*', '*', '*'], ['a2', '*', 'c3', 'd4'], ['a2', '*', 'c3', 'd4']]


In [37]:
# Compressed base table
lData1 = tuple_convert(lData1)
dFeatures = generate_compressed_table(lData1)
print("Compressed base table:-")
print(dFeatures)

Compressed base table:-
{('a1', 'b1', '*', '*'): 2, ('a1', '*', '*', '*'): 1, ('a2', '*', 'c3', 'd4'): 2}


In [39]:
lKeys = lexographical_Sort_keys(dFeatures)
star_tree = create_star_tree(lKeys, dFeatures)
lCuboidValList = ["*"] * (len(lData[0]) + 1)
lCuboidValList[0] = star_tree.count
lSubTrees = ["*"] * (len(lData[0]) + 1)
starcubing(star_tree, star_tree, 0, lCuboidValList, dFeatures, lSubTrees)
print(lSubTrees)

[5, 3, '*', '*', '*']
[5, 3, 3, '*', '*']
[5, 3, 3, 2, '*']
[5, 3, 3, 2, 2]
[5, 5, '*', '*', '*']
[5, 5, 2, '*', '*']
[5, 5, 2, 2, '*']
[5, 5, 2, 2, 2]
[5, '*', '*', '*', '*']
[<__main__.Tree object at 0x000001A196065358>, <__main__.Tree object at 0x000001A196065198>, <__main__.Tree object at 0x000001A196065208>, <__main__.Tree object at 0x000001A196065278>, <__main__.Tree object at 0x000001A1960652E8>]
